In [3]:
# 필수 라이브러리 설치
#%pip install pydantic

from dotenv import load_dotenv
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv()

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser

from pydantic import BaseModel, Field
from typing import List

In [4]:
# 출력 구조를 정의하는 Pydantic 모델
class IntroduceFormat(BaseModel):
    name: str = Field(description="이름")
    age: int = Field(description="나이")
    major: str = Field(description="전공")
    habit: List[str] = Field(description="3가지 취미")
    goal: str = Field(description="목표")
    
# Pydantic 출력 파서 초기화
parser = PydanticOutputParser(pydantic_object=IntroduceFormat)

# 프롬프트 템플릿 설정
template = """
다음 학생 사용자의 자유 형식 자기소개를 입력받아 이름, 나이, 전공, 취미리스트, 목표를 구조화된 형태로 출력해주세요.

출력 형식 예시는 다음과 같습니다.
{{
    "name": "김민수",
    "age": 22,
    "major": "컴퓨터공학",
    "hobbies": ["게임하기", "영화보기", "코딩"],
    "goal": "훌륭한 개발자가 되는 것"
}}
요청: {query}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template)

# 파서의 지시사항을 프롬프트에 주입
prompt = prompt.partial(
    format_instructions=parser.get_format_instructions()
)

# ChatOpenAI 모델 초기화
model = ChatOpenAI(temperature=0.7, model="gpt-3.5-turbo")

# 체인 구성 및 실행
query = input()
chain = prompt | model | parser
output = chain.invoke({"query": query})

# 결과 출력
print(f"이름: {output.name}")
print(f"나이: {output.age}")
print(f"전공: {output.major}")
print(f"취미 리스트: {', '.join(output.habit)}")
print(f"목표: {output.goal}")

이름: 김민수
나이: 22
전공: 컴퓨터공학
취미 리스트: 게임하기, 영화보기, 코딩
목표: 훌륭한 개발자가 되는 것
